In [1]:
import os, sys, random, shutil
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
from shutil import copyfile
import pandas as pd
from sklearn import preprocessing, model_selection
import matplotlib.pyplot as plt
#%matplotlib inline
from matplotlib import patches
import numpy as np
import cv2

In [2]:
#from google.colab import drive
#drive.mount('/content/gdrive')

### Extracting information from Tobacco-800 XML files

In [3]:
df = []
cnt = 0

# refer the xml files to understand its structure and revist this code block.
annotations = sorted(glob('Input data/Tobacc800_Groundtruth_v2.0/XMLGroundtruth_v2.0/*.xml'))
for file in annotations:
    myroot = ET.parse(file).getroot()
    # image filename is changed aah97e00-page02_1.tif -> 0.tif, so the previous filename is collected here.
    prev_filename = myroot[0].attrib['src']
    filename = str(cnt) + '.tif' # new filename based on the count (0.tif, 1.tif etc)
    page_height, page_width =  myroot[0][0].attrib['height'], myroot[0][0].attrib['width']
    
    row = []
    # An image might have multiple items (zones) (logos and signs), so iterate through each zones
    for zone in myroot[0][0]: 
        category = zone.attrib['gedi_type'] # type of zone (DLLogo/ DLSignature)
        id = zone.attrib['id']
        x, y = zone.attrib['col'], zone.attrib['row'] # x, y coordinate
        w, h = zone.attrib['width'], zone.attrib['height'] # width and height of bbox
        
        # Signature have Authors, represeting whose signature it is
        if category == 'DLSignature':
            AuthorID = zone.attrib['AuthorID']
            Overlapped = zone.attrib['Overlapped']
        else:
            # Logos don't have authors.
            AuthorID, Overlapped = ('NA', 'NA')
        row = [prev_filename, filename, page_height, page_width, AuthorID, Overlapped, category, id, x, y, w, h]
        df.append(row)
    cnt += 1

### prova myroot function

In [4]:
# https://www.datacamp.com/tutorial/python-xml-elementtree
print(ET.tostring(myroot, encoding='utf8').decode('utf8'))

<?xml version='1.0' encoding='utf8'?>
<ns0:GEDI xmlns:ns0="http://lamp.cfar.umd.edu/GEDI" version="1.0">
	<ns0:DL_DOCUMENT src="zvs17e00.tif" NrOfPages="1" docTag="xml">
		<ns0:DL_PAGE gedi_type="DL_PAGE" src="zvs17e00.tif" pageID="1" width="2560" height="3295">
			<ns0:DL_ZONE gedi_type="DLSignature" id="None" col="1649" row="2094" width="542" height="167" Overlapped="Yes" AuthorID="Landry, J.T."> </ns0:DL_ZONE>
		</ns0:DL_PAGE>
	</ns0:DL_DOCUMENT>
</ns0:GEDI>


In [5]:
#myroot = ET.parse(file).getroot()
tree = ET.parse('Input data/Tobacc800_Groundtruth_v2.0/XMLGroundtruth_v2.0/zvs17e00.xml')
myroot = tree.getroot()
#GEDI is the single root element that contains all the other elements

In [6]:
print(myroot[0][0])

<Element '{http://lamp.cfar.umd.edu/GEDI}DL_PAGE' at 0x000001A6241817C0>


In [7]:
prev_filename = myroot[0].attrib['src']
filename = str(cnt) + '.tif'
page_height =  myroot[0][0].attrib['height']
page_width =  myroot[0][0].attrib['width']
print(prev_filename, filename, page_height, page_width)

zvs17e00.tif 1290.tif 3295 2560


In [8]:
for child in myroot:
    print(myroot.tag, myroot.attrib)

{http://lamp.cfar.umd.edu/GEDI}GEDI {'version': '1.0'}


In [9]:
#all the elements in the tree
[elem.tag for elem in myroot.iter()]

['{http://lamp.cfar.umd.edu/GEDI}GEDI',
 '{http://lamp.cfar.umd.edu/GEDI}DL_DOCUMENT',
 '{http://lamp.cfar.umd.edu/GEDI}DL_PAGE',
 '{http://lamp.cfar.umd.edu/GEDI}DL_ZONE']

In [10]:
for zone in myroot[0][0]: 
        category = zone.attrib['gedi_type'] # type of zone (DLLogo/ DLSignature)
        id = zone.attrib['id']
        x, y = zone.attrib['col'], zone.attrib['row'] # x, y coordinate
        w, h = zone.attrib['width'], zone.attrib['height'] # width and height of bbox

print(category, id, x, y, w, h)

DLSignature None 1649 2094 542 167


### fine prova

In [11]:
data = pd.DataFrame(df, columns=['prev_filename', 'filename', 'page_height', 'page_width', 'AuthorID', 'Overlapped', 'category', 'id', 'x', 'y', 'width','height'])
data

,prev_filename,filename,page_height,page_width,AuthorID,Overlapped,category,id,x,y,width,height
0,aah97e00-page02_1.tif,0.tif,3296,2560,NA,NA,DLLogo,None,1074,18,374,219
1,aah97e00-page02_2.tif,1.tif,3296,2560,"Boder, J.B.",Yes,DLSignature,None,1409,793,659,361
2,aam09c00.tif,2.tif,2292,1728,"Koplow, M.G.",Yes,DLSignature,None,821,1422,757,183
3,aao54e00_2.tif,4.tif,3296,2560,"Landry, J.T.",Yes,DLSignature,None,1514,708,627,206
4,abm69c00.tif,5.tif,2292,1728,"Feldman, J.D.",Yes,DLSignature,None,667,1279,617,110
...,...,...,...,...,...,...,...,...,...,...,...,...
1337,zrz94a00-page02_1.tif,1284.tif,2156,1728,NA,NA,DLLogo,None,737,116,191,124
1338,zrz94a00-page02_2.tif,1285.tif,2156,1728,"Dimech, Ellen",Yes,DLSignature,None,828,1046,362,91
1339,zss86d00.tif,1286.tif,2292,1728,"Nielsen, Vigo G.",Yes,DLSignature,None,867,1734,363,189
1340,ztz52d00-page02_2.tif,1288.tif,2292,1728,"Bergman, J.I.",Yes,DLSignature,None,896,651,205,178


In [12]:
data1=data.query("prev_filename == 'abm69c00.tif'")
data1.groupby(['prev_filename']).count()

,filename,page_height,page_width,AuthorID,Overlapped,category,id,x,y,width,height
prev_filename,,,,,,,,,,,
abm69c00.tif,1,1,1,1,1,1,1,1,1,1,1


In [13]:
data.groupby(['category']).count()

,prev_filename,filename,page_height,page_width,AuthorID,Overlapped,id,x,y,width,height
category,,,,,,,,,,,
DLLogo,432,432,432,432,432,432,432,432,432,432,432
DLSignature,910,910,910,910,910,910,910,910,910,910,910


In [14]:
test = data[['page_height', 'page_width']]
test.max(), test.min()

(page_height    3584
 page_width     2720
 dtype: object,
 page_height    1575
 page_width     1200
 dtype: object)

### Scaling the image to reduce training time

In [15]:
BASE_DIR = 'Input data/Tobacco800_SinglePage/SinglePageTIF/'
SAVE_PATH = 'Input data/New format/Data Scaled/'

isExist = os.path.exists(SAVE_PATH)

if not isExist:
  os.mkdir(SAVE_PATH)
else:
  pass

FileNotFoundError: [WinError 3] Impossibile trovare il percorso specificato: 'Input data/New format/Data Scaled/'

In [ ]:
def scale_image(df):
    df_new = []
    file_name = data['prev_filename']
    X, Y, W, H = map(int, data.x), map(int, data.y), map(int, data.width), map(int, data.height)
    for file, x, y, w, h in zip(file_name, X, Y, W, H):
        image_path = BASE_DIR + file
        img = cv2.imread(image_path) #return a numpy array
        page_height, page_width = img.shape[:2]
        max_height = 640
        max_width = 480

        # computes the scaling factor
        if max_height < page_height or max_width < page_width:
            scaling_factor = max_height / float(page_height)
            if max_width/float(page_width) < scaling_factor:
                scaling_factor = max_width / float(page_width)
            # scale the image with the scaling factor
            img = cv2.resize(img, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
            #INTER_AREA: method that decides which pixel gets which value based on its neighboring pixels and the scale at which the image is being resized.
        jpg_filename = file[:-4] + '.jpg'
        new_file_path = SAVE_PATH + jpg_filename
        cv2.imwrite(new_file_path, img) # write the scales image

        # save new page height and width
        page_height, page_width = page_height*scaling_factor, page_width*scaling_factor
        # compute new x, y, w, h coordinates after scaling
        x, y, w, h= int(x*scaling_factor), int(y*scaling_factor), int(w*scaling_factor), int(h*scaling_factor)
        row = [jpg_filename, x, y, w, h, page_height, page_width]
        df_new.append(row)
    return df_new
scaled_data = scale_image(data)

### Adding the information regarding the scaling to the df

In [ ]:
scaled_data = list(zip(*scaled_data))

data['new_filename'] = scaled_data[0]
data['x_scaled'] = scaled_data[1]
data['y_scaled'] = scaled_data[2]
data['w_scaled'] = scaled_data[3]
data['h_scaled'] = scaled_data[4]
data['page_height_scaled'] = scaled_data[5]
data['page_width_scaled'] = scaled_data[6]
data.head(10)

### Testing the scaled image

In [ ]:
img = cv2.imread('Input data/New format/Data Scaled/aah97e00-page02_2.jpg')

img = cv2.rectangle(img, (264, 148), (264+123, 148+67), (255, 0, 0), 1)
#cv2.rectangle(image, start_point, end_point, color, thickness)
plt.figure(figsize=(6, 6))
plt.imshow(img)

In [ ]:
df2=data.query("new_filename == 'aao54e00_2.jpg'")
df2

In [ ]:
df2.groupby(['new_filename']).count()

### Saving to CSV file

In [ ]:
isExist = os.path.exists('Input data/New format/Data Cleaned/')

if not isExist:
  os.mkdir('Input data/New format/Data Cleaned/')
else:
  pass

In [ ]:
data[['prev_filename', 'new_filename', 'filename', 'page_height', 'page_width', 'page_height_scaled', 'page_width_scaled', 'AuthorID', 'Overlapped', 'category', 'id', 'x', 'y', 'width', 'height', 'x_scaled', 'y_scaled', 'w_scaled', 'h_scaled']].to_csv('Input data/New format/Data Cleaned/data_cleaned.csv', index=False)
data.head(3)

### Converting data to YOLOv5 format

In [ ]:
def x_center(df):
  return int(df.x_scaled + (df.w_scaled/2))
def y_center(df):
  return int(df.y_scaled + (df.h_scaled/2))

def w_norm(df, col):
  return df[col]/df['page_width_scaled']
def h_norm(df, col):
  return df[col]/df['page_height_scaled']

df = pd.read_csv('Input data/New format/Data Cleaned/data_cleaned.csv')

le = preprocessing.LabelEncoder() #used to normalize labels
le.fit(df['category'])
print(le.classes_)
labels = le.transform(df['category'])
df['labels'] = labels #0,1 depending on the category (logo = 0, signature = 1)


df['x_center'] = df.apply(x_center, axis=1)
df['y_center'] = df.apply(y_center, axis=1)

df['x_center_norm'] = df.apply(w_norm, col='x_center',axis=1)
df['width_norm'] = df.apply(w_norm, col='w_scaled', axis=1)

df['y_center_norm'] = df.apply(h_norm, col='y_center',axis=1)
df['height_norm'] = df.apply(h_norm, col='h_scaled',axis=1)

df.head()

In [ ]:
df.to_csv('final_input_data.csv')

### Moving images to train and valid folders

In [ ]:
df_train, df_valid = model_selection.train_test_split(df, test_size=0.1, random_state=13, shuffle=True)
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html 
print(df_train.shape, df_valid.shape)

Creating relevant directories

In [ ]:
path = 'Input data/New format/'
directories = ['images/', 'images/train/', 'images/valid/', 'labels/', 'labels/train/', 'labels/valid/']

for i in directories:
  new_path = path+i
  isExist = os.path.exists(new_path)
  if not isExist:
    os.mkdir(new_path)
  else:
    pass

Segregating images and labels to train and valid

In [ ]:
def segregate_data(df, img_path, label_path, train_img_path, train_label_path):
  filenames = []
  for filename in df.filename:
    filenames.append(filename)
  filenames = set(filenames)
  
  for filename in filenames:
    yolo_list = []

    for _,row in df[df.filename == filename].iterrows():
      yolo_list.append([row.labels, row.x_center_norm, row.y_center_norm, row.width_norm, row.height_norm])

    yolo_list = np.array(yolo_list)
    txt_filename = os.path.join(train_label_path,str(row.new_filename.split('.')[0])+".txt") #prende il path labels train e la stringa del nome del file.txt
    # Save the .img & .txt files to the corresponding train and validation folders
    np.savetxt(txt_filename, yolo_list, fmt=["%d", "%f", "%f", "%f", "%f"]) #%d To save as an integer, %f To save as a float, %s To save as a string
          #yolo_list is: data to be saved to a text file
    shutil.copyfile(os.path.join(img_path,row.new_filename), os.path.join(train_img_path,row.new_filename)) #used to copy the content of the source file to the destination file

In [ ]:
df.filename

In [ ]:
# Apply function
src_img_path = "Input data/New format/Data Scaled/"
src_label_path = "Input data/Tobacc800_Groundtruth_v2.0/"

In [ ]:
train_img_path = "Input data/New format/images/train"
train_label_path = "Input data/New format/labels/train"

valid_img_path = "Input data/New format/images/valid"
valid_label_path = "Input data/New format/labels/valid"

segregate_data(df_train, src_img_path, src_label_path, train_img_path, train_label_path)
segregate_data(df_valid, src_img_path, src_label_path, valid_img_path, valid_label_path)

print("No. of Training images", len(os.listdir('Input data/New format/images/train')))
print("No. of Training labels", len(os.listdir('Input data/New format/labels/train')))

print("No. of valid images", len(os.listdir('Input data/New format/images/valid')))
print("No. of valid labels", len(os.listdir('Input data/New format/labels/valid')))

Deleting ipython checkpoints

In [ ]:
try:
  shutil.rmtree('Input data/New format/images/train/.ipynb_checkpoints') #delete an entire directory tree
except FileNotFoundError:
  pass

try:
  shutil.rmtree('Input data/New format/images/valid/.ipynb_checkpoints')
except FileNotFoundError:
  pass

try:
  shutil.rmtree('Input data/New format/labels/train/.ipynb_checkpoints')
except FileNotFoundError:
  pass

try:
  shutil.rmtree('Input data/New format/labels/valid/.ipynb_checkpoints')
except FileNotFoundError:
  pass